In [1]:
#!wget -O pose_models/pose_landmarker_lite.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/1/pose_landmarker_lite.task
#!wget -O pose_models/pose_landmarker_full.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/1/pose_landmarker_full.task
#!wget -O pose_models/pose_landmarker_heavy.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

In [2]:
# Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pandas as pd
import os

#### Extract the landmarks for training data

In [3]:
# Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='../models/pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Sets the result listener to receive the landmarker results asynchronously when Pose Landmarker is in the live stream mode. Can only be used when running mode is set to LIVE_STREAMLoop through all the images in the 'data/img/' directory.
img_dir = "../data/img/"
output_dir = "../data/csv/"

for img_file in os.listdir(img_dir):
    if img_file.endswith(".jpg"):
        # STEP 4: Load the input image.
        image = mp.Image.create_from_file(os.path.join(img_dir, img_file))
        
        # STEP 5: Detect pose landmarks from the input image.
        detection_result = detector.detect(image)
        
        # Convert pose_landmarks to a list of dictionaries
        data = []
        for pose_landmarks in detection_result.pose_landmarks:
            landmarks_data = []
            for pose_landmark in pose_landmarks:
                landmarks_data.append({
                    'x': pose_landmark.x,
                    'y': pose_landmark.y,
                    'z': pose_landmark.z,
                    'visibility': pose_landmark.visibility,
                    'presence': pose_landmark.presence
                })
            data.extend(landmarks_data)
        
        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(data)
        
        # Save the DataFrame as a CSV file
        csv_filename = os.path.splitext(img_file)[0] + ".csv"
        df.to_csv(os.path.join(output_dir, csv_filename), index=False)

I0000 00:00:1714405346.493764   47816 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1714405346.496810   47877 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.6-arch1.2), renderer: Mesa Intel(R) HD Graphics 620 (KBL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
